In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib as plt

In [4]:
fec = pd.read_csv('E:/datasets/fec/P00000001-ALL.csv')

C:\Users\gg246\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
fec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001731 entries, 0 to 1001730
Data columns (total 16 columns):
cmte_id              1001731 non-null object
cand_id              1001731 non-null object
cand_nm              1001731 non-null object
contbr_nm            1001731 non-null object
contbr_city          1001712 non-null object
contbr_st            1001727 non-null object
contbr_zip           1001620 non-null object
contbr_employer      988002 non-null object
contbr_occupation    993301 non-null object
contb_receipt_amt    1001731 non-null float64
contb_receipt_dt     1001731 non-null object
receipt_desc         14166 non-null object
memo_cd              92482 non-null object
memo_text            97770 non-null object
form_tp              1001731 non-null object
file_num             1001731 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 122.3+ MB


In [6]:
unique_cands=fec.cand_nm.unique()

In [7]:
unique_cands

array(['Bachmann, Michelle', 'Romney, Mitt', 'Obama, Barack',
       "Roemer, Charles E. 'Buddy' III", 'Pawlenty, Timothy',
       'Johnson, Gary Earl', 'Paul, Ron', 'Santorum, Rick',
       'Cain, Herman', 'Gingrich, Newt', 'McCotter, Thaddeus G',
       'Huntsman, Jon', 'Perry, Rick'], dtype=object)

In [11]:
parties={'Bachmann, Michelle':'Repub', 
         'Romney, Mitt':'Repub', 
         'Obama, Barack':'Demo',
       "Roemer, Charles E. Buddy' III":'Repub',
         'Pawlenty, Timothy':'Repub',
       'Johnson, Gary Earl':'Repub',
         'Paul, Ron':'Repub',
         'Santorum, Rick':'Repub',
       'Cain, Herman':'Repub',
         'Gingrich, Newt':'Repub', 
         'McCotter, Thaddeus G':'Repub',
       'Huntsman, Jon':'Repub', 
         'Perry, Rick':'Repub'}

In [12]:
fec.cand_nm[123456:123461]

123456    Obama, Barack
123457    Obama, Barack
123458    Obama, Barack
123459    Obama, Barack
123460    Obama, Barack
Name: cand_nm, dtype: object

In [13]:
fec.cand_nm[123456:123461].map(parties)

123456    Demo
123457    Demo
123458    Demo
123459    Demo
123460    Demo
Name: cand_nm, dtype: object

In [14]:
fec['party']=fec.cand_nm.map(parties)

In [16]:
fec['party'].value_counts()

Demo     593746
Repub    402065
Name: party, dtype: int64

In [17]:
(fec.contb_receipt_amt>0).value_counts()

True     991475
False     10256
Name: contb_receipt_amt, dtype: int64

In [18]:
fec = fec[fec.contb_receipt_amt>0]

In [76]:
fec_m=fec[fec.cand_nm.isin(['Obama, Barack','Romney, Mitt'])]


In [30]:
occ_mapping={
    'INFORMATION REQUESTED PER BEST EFFORTS':'NOT PROVIED',
    'INFORMATION REQUESTED':'NOT PROVIED',
    'INFORMATION REQUESTED (BEST EFFORTS)':'NOT PROVIED',
    'C.E.O.':'CEO'
}
f = lambda x: occ_mapping.get(x,x)
fec.contbr_occupation=fec.contbr_occupation.map(f)

In [23]:
fec.contbr_occupation.value_counts()[:10]

RETIRED                                   233990
INFORMATION REQUESTED                      35107
ATTORNEY                                   34286
HOMEMAKER                                  29931
PHYSICIAN                                  23432
INFORMATION REQUESTED PER BEST EFFORTS     21138
ENGINEER                                   14334
TEACHER                                    13990
CONSULTANT                                 13273
PROFESSOR                                  12555
Name: contbr_occupation, dtype: int64

In [32]:
fec.contbr_employer.value_counts()[:10]


RETIRED                                   206675
SELF-EMPLOYED                              94505
NOT EMPLOYED                               45877
INFORMATION REQUESTED                      36135
SELF                                       24385
INFORMATION REQUESTED PER BEST EFFORTS     22260
NONE                                       19929
HOMEMAKER                                  18269
SELF EMPLOYED                               6274
REQUESTED                                   4233
Name: contbr_employer, dtype: int64

In [35]:
emp={
    'INFORMATION REQUESTED PER BEST EFFORTS':'NOT PROVIED',
    'INFORMATION REQUESTED ':'NOT PROVIED',
    'SELF':'SELF-EMPLOYED',
    'SELF EMPLOYED':'SELF-EMPLOYED'
}
f = lambda x:emp.get(x,x)
fec.contbr_employer=fec.contbr_employer.map(f)

In [39]:
by_occ = fec.pivot_table('contb_receipt_amt',index='contbr_occupation',columns='party',aggfunc='sum')

In [40]:
over_2m = by_occ[by_occ.sum(1)>2000000]

In [41]:
over_2m

party,Demo,Repub
contbr_occupation,,
ATTORNEY,11141982.97,7.462058e+06
CEO,2074974.79,4.208951e+06
CONSULTANT,2459912.71,2.538990e+06
ENGINEER,951525.55,1.811937e+06
EXECUTIVE,1355161.05,4.136400e+06
HOMEMAKER,4248875.80,1.362560e+07
INVESTOR,884133.00,2.431259e+06
LAWYER,3160478.87,3.911243e+05
MANAGER,762883.22,1.441092e+06


In [44]:
over_2m.plot(kind='barh')

In [43]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [58]:
def get_top(group,key,n=5):
    totals = group.groupby(key)['contb_receipt_amt'].sum()
    return totals.nlargest(n)

In [60]:
grouped = fec_m.groupby('cand_nm')

In [65]:
grouped.apply(get_top,'contbr_occupation',n=100)

contbr_occupation,RETIRED,INFORMATION REQUESTED PER BEST EFFORTS,HOMEMAKER,ATTORNEY,PRESIDENT,EXECUTIVE,C.E.O.,INVESTOR,CONSULTANT,PHYSICIAN,...,RANCHER,OFFICE MANAGER,MONEY MANAGER,INVESTMENT PROFESSIONAL,CHAIRMAN OF THE BOARD,BROKER,INSURANCE AGENT,FINANCIAL ANALYST,PROPERTY MANAGER,CONSULTING
cand_nm,,,,,,,,,,,,,,,,,,,,,
"Romney, Mitt",11508473.59,11396894.84,8147446.22,5364718.82,2491244.89,2300947.03,1968386.11,1537595.12,1424894.01,1368023.96,...,81717.0,78585.5,78250.0,77450.0,74600.0,74347.0,70696.0,69143.0,69101.12,68868.12


In [59]:
bins = np.array([0,1,10,100,1000,10000,100000,1000000,10000000])

In [77]:
labels=pd.cut(fec_m.contb_receipt_amt,bins)

In [78]:
labels

411         (10, 100]
412       (100, 1000]
413       (100, 1000]
414         (10, 100]
415         (10, 100]
             ...     
701381      (10, 100]
701382    (100, 1000]
701383        (1, 10]
701384      (10, 100]
701385    (100, 1000]
Name: contb_receipt_amt, Length: 694282, dtype: category
Categories (8, interval[int64]): [(0, 1] < (1, 10] < (10, 100] < (100, 1000] < (1000, 10000] < (10000, 100000] < (100000, 1000000] < (1000000, 10000000]]

In [79]:
grouped=fec_m.groupby(['cand_nm',labels])

In [80]:
grouped.size().unstack(0)

cand_nm,"Obama, Barack","Romney, Mitt"
contb_receipt_amt,,
"(0, 1]",493.0,77.0
"(1, 10]",40070.0,3681.0
"(10, 100]",372280.0,31853.0
"(100, 1000]",153991.0,43357.0
"(1000, 10000]",22284.0,26186.0
"(10000, 100000]",2.0,1.0
"(100000, 1000000]",3.0,NaN
"(1000000, 10000000]",4.0,NaN


In [81]:
buk_sums = grouped.contb_receipt_amt.sum().unstack(0)

In [83]:
norm_sums = buk_sums.div(buk_sums.sum(axis=1),axis=0)

In [84]:
norm_sums

cand_nm,"Obama, Barack","Romney, Mitt"
contb_receipt_amt,,
"(0, 1]",0.805182,0.194818
"(1, 10]",0.918767,0.081233
"(10, 100]",0.910769,0.089231
"(100, 1000]",0.710176,0.289824
"(1000, 10000]",0.447326,0.552674
"(10000, 100000]",0.823120,0.176880
"(100000, 1000000]",1.000000,NaN
"(1000000, 10000000]",1.000000,NaN


In [85]:
norm_sums[:-2].plot(kind='barh')

In [87]:
group =fec_m.groupby(['cand_nm','contbr_st'])

In [88]:
totals = group.contb_receipt_amt.sum().unstack(0).fillna(0)

In [89]:
totals = totals[totals.sum(1)>100000]

In [90]:
totals[:10]

cand_nm,"Obama, Barack","Romney, Mitt"
contbr_st,,
AK,281840.15,86204.24
AL,543123.48,527303.51
AR,359247.28,105556.00
AZ,1506476.98,1888436.23
CA,23824984.24,11237636.60
CO,2132429.49,1506714.12
CT,2068291.26,3499475.45
DC,4373538.80,1025137.50
DE,336669.14,82712.00


In [92]:
percent = totals.div(totals.sum(1), axis=0)

In [93]:
percent[:10]

cand_nm,"Obama, Barack","Romney, Mitt"
contbr_st,,
AK,0.765778,0.234222
AL,0.507390,0.492610
AR,0.772902,0.227098
AZ,0.443745,0.556255
CA,0.679498,0.320502
CO,0.585970,0.414030
CT,0.371476,0.628524
DC,0.810113,0.189887
DE,0.802776,0.197224
